# IBM Applied Data Science Capstone Course by Coursera

## Week 5 Final Report

### 1. Import libraries

In [4]:
#importing libraries
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### 2. Scrap data from Wikipedia page into a DataFrame

In [5]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Jakarta").text

In [6]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [7]:
# create a list to store neighborhood data
neighborhoodList = []

In [8]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [9]:
# create a new DataFrame from the list
jkt_df = pd.DataFrame({"Neighborhood": neighborhoodList})

jkt_df.head()

,Neighborhood
0,List of districts of Jakarta
1,List of Jakarta Subdistricts
2,Cakung
3,Cempaka Putih
4,Cengkareng


In [10]:
#Drop the first two rows
jkt_df=jkt_df.loc[2:45]
jkt_df.index=range(len(jkt_df))
jkt_df

,Neighborhood
0,Cakung
1,Cempaka Putih
2,Cengkareng
3,Cilandak
4,Cilincing
5,Cipayung
6,Ciracas
7,Duren Sawit
8,"Gambir, Jakarta"
9,Grogol Petamburan


In [11]:
# print the number of rows of the dataframe
jkt_df.shape

(44, 1)

### 3. Get the geographical coordinates

In [12]:
# define a function to get coordinates (latitude and longitude)
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Jakarta, Indonesia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in jkt_df["Neighborhood"].tolist() ]

In [14]:
coords

[[-6.196229999999957, 106.93522000000007],
 [-6.175999999999931, 106.87061000000006],
 [-6.1305999999999585, 106.74559000000005],
 [-6.2905099999999265, 106.79491000000007],
 [-6.113579999999956, 106.94911000000008],
 [-6.326729999999941, 106.90298000000007],
 [-6.3287899999999695, 106.88475000000005],
 [-6.2350099999999316, 106.92261000000008],
 [-6.17298999999997, 106.81571000000008],
 [-6.167769999999962, 106.78460000000007],
 [-6.343079999999929, 106.81745000000006],
 [-6.232649999999978, 106.88539000000009],
 [-6.152889999999957, 106.78435000000007],
 [-6.1858799999999405, 106.85598000000005],
 [-6.259049999999945, 106.84190000000007],
 [-6.122099999999932, 106.70727000000005],
 [-6.251469999999927, 106.79754000000008],
 [-6.263859999999966, 106.77519000000007],
 [-6.197019999999952, 106.77308000000005],
 [-6.16020999999995, 106.90478000000007],
 [-6.165839999999946, 106.85396000000009],
 [-6.218229999999949, 106.73749000000004],
 [-6.127159999999947, 106.89919000000003],
 [-6.270

In [15]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
# merge the coordinates into the original dataframe
jkt_df['Latitude'] = df_coords['Latitude']
jkt_df['Longitude'] = df_coords['Longitude']

In [17]:
# check the neighborhoods and the coordinates
print(jkt_df.shape)
jkt_df

(44, 3)


,Neighborhood,Latitude,Longitude
0,Cakung,-6.196230,106.935220
1,Cempaka Putih,-6.176000,106.870610
2,Cengkareng,-6.130600,106.745590
3,Cilandak,-6.290510,106.794910
4,Cilincing,-6.113580,106.949110
5,Cipayung,-6.326730,106.902980
6,Ciracas,-6.328790,106.884750
7,Duren Sawit,-6.235010,106.922610
8,"Gambir, Jakarta",-6.172990,106.815710
9,Grogol Petamburan,-6.167770,106.784600


In [35]:
# save the DataFrame as CSV file
jkt_df.to_csv("jkt_df.csv", index=False)

### 4. Create a map of Jakarta with neighborhoods superimposed on top

In [18]:
# get the coordinates of Jakarta
address = 'Jakarta, Indonesia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Jakarta, Indonesia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Jakarta, Indonesia -6.1753942, 106.827183.


In [19]:
# create map of Jakarta using latitude and longitude values
map_jkt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(jkt_df['Latitude'], jkt_df['Longitude'], jkt_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_jkt)  
    
map_jkt

In [38]:
# save the map as HTML file
map_jkt.save('map_jkt.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'D2JTKE0FPHEEBVLX4CWTCMD4QBVEHCPNJGTRM4JKIDVRQKV0' # your Foursquare ID
CLIENT_SECRET = '3SF5B3TPBC5XRHBCGNC2PVAGLR1FPMTI33X2PJQCLE5XKC0R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D2JTKE0FPHEEBVLX4CWTCMD4QBVEHCPNJGTRM4JKIDVRQKV0
CLIENT_SECRET:3SF5B3TPBC5XRHBCGNC2PVAGLR1FPMTI33X2PJQCLE5XKC0R


### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [21]:
radius = 2000
LIMIT = 100


venues = []

for lat, long, neighborhood in zip(jkt_df['Latitude'], jkt_df['Longitude'], jkt_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3578, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cakung,-6.19623,106.93522,Bakmi Akwet Taman Modern,-6.190151,106.948378,Chinese Restaurant
1,Cakung,-6.19623,106.93522,Starbucks,-6.186618,106.948836,Coffee Shop
2,Cakung,-6.19623,106.93522,Lulu Hypermarket,-6.186926,106.949000,Supermarket
3,Cakung,-6.19623,106.93522,Pulo Gebang Permai,-6.197495,106.949006,Department Store
4,Cakung,-6.19623,106.93522,Imperial Kitchen & Dimsum,-6.187565,106.950250,Dim Sum Restaurant


### Let's check how many venues were returned for each neighorhood

In [45]:
venues_df["Neighborhood"].groupby(venues_df["Neighborhood"]).agg('count')

Neighborhood
Cakung                          24
Cempaka Putih                   84
Cengkareng                      68
Cilandak                       100
Cilincing                        5
Cipayung                        20
Ciracas                         32
Duren Sawit                     75
Gambir, Jakarta                100
Grogol Petamburan              100
Jagakarsa                       50
Jatinegara                      81
Jelambar                       100
Johar Baru                     100
Kalibata                       100
Kalideres                       45
Kebayoran Baru                 100
Kebayoran Lama                 100
Kebon Jeruk                    100
Kelapa Gading                  100
Kemayoran                      100
Kembangan, Jakarta              34
Koja, Jakarta                   41
Kramat Jati, East Jakarta       48
Makasar, Jakarta                64
Mampang Prapatan               100
Matraman                       100
Menteng                        100
Pademan

### Let's find out how many unique categories can be curated from all the returned venues

In [46]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 259 uniques categories.


In [47]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Chinese Restaurant', 'Coffee Shop', 'Department Store',
       'Supermarket', 'Dim Sum Restaurant', 'Clothing Store',
       'Gas Station', 'Resort', 'Donut Shop', 'Food Truck', 'Bookstore',
       'Fast Food Restaurant', 'Diner', 'Lounge', 'Pizza Place', 'Arcade',
       'Seafood Restaurant', 'Sculpture Garden', 'Indonesian Restaurant',
       'Neighborhood', 'Hotel Bar', 'Food', 'Padangnese Restaurant',
       'Soup Place', 'Café', 'Athletics & Sports', 'Bistro',
       'Farmers Market', 'Japanese Restaurant', 'Bakery',
       'Acehnese Restaurant', 'Multiplex', 'Shopping Mall', 'Hotel',
       'Housing Development', 'Restaurant', 'Korean Restaurant',
       'Noodle House', 'Tanning Salon', 'Juice Bar', 'Flea Market',
       'Comedy Club', 'Middle Eastern Restaurant', 'Asian Restaurant',
       "Doctor's Office", 'Nightclub', 'Wings Joint',
       'Salon / Barbershop', 'Residential Building (Apartment / Condo)',
       'BBQ Joint', 'Theme Park', 'Convenience Store', 'Pool',
 

In [48]:
coffeeshop_df=venues_df[venues_df.VenueCategory=="Coffee Shop"]
cafe_df=venues_df[venues_df.VenueCategory=="Café"]

In [49]:
coffeeshop_df

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
1,Cakung,-6.19623,106.93522,Starbucks,-6.186618,106.948836,Coffee Shop
28,Cempaka Putih,-6.17600,106.87061,Starbucks,-6.177065,106.875270,Coffee Shop
37,Cempaka Putih,-6.17600,106.87061,Kopi Oey,-6.176534,106.872551,Coffee Shop
39,Cempaka Putih,-6.17600,106.87061,Starbucks,-6.188838,106.874354,Coffee Shop
42,Cempaka Putih,-6.17600,106.87061,Starbucks,-6.169607,106.868399,Coffee Shop
47,Cempaka Putih,-6.17600,106.87061,Starbucks,-6.171625,106.863239,Coffee Shop
76,Cempaka Putih,-6.17600,106.87061,Anastomosis Coffee,-6.185540,106.884660,Coffee Shop
78,Cempaka Putih,-6.17600,106.87061,Coffee Toffee,-6.166886,106.875265,Coffee Shop
118,Cengkareng,-6.13060,106.74559,Starbucks,-6.123781,106.739900,Coffee Shop
142,Cengkareng,-6.13060,106.74559,Starbucks,-6.122304,106.731144,Coffee Shop


### 6. Analyze Each Neighborhood

In [23]:
# one hot encoding
jkt_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jkt_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jkt_onehot.columns[-1]] + list(jkt_onehot.columns[:-1])
jkt_onehot = jkt_onehot[fixed_columns]

print(jkt_onehot.shape)
jkt_onehot.head()

(3578, 263)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,...,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Wine Bar,Wings Joint,Women's Store,Zoo
0,Cakung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cakung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cakung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cakung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cakung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
jkt_grouped = jkt_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(jkt_grouped.shape)
jkt_grouped

(44, 263)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,...,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Wine Bar,Wings Joint,Women's Store,Zoo
0,Cakung,0.00,0.000000,0.00,0.00,0.000000,0.00,0.064516,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
1,Cempaka Putih,0.00,0.012195,0.00,0.00,0.000000,0.00,0.012195,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.012195,0.00,0.000000
2,Cengkareng,0.00,0.000000,0.00,0.00,0.015152,0.00,0.000000,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.015152,0.00,0.000000,0.00,0.000000
3,Cilandak,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,...,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.00,0.000000
4,Cilincing,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
5,Cipayung,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
6,Ciracas,0.00,0.000000,0.00,0.00,0.000000,0.00,0.032258,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.032258,0.00,0.000000,0.00,0.000000
7,Duren Sawit,0.00,0.000000,0.00,0.00,0.000000,0.00,0.012500,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
8,"Gambir, Jakarta",0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,...,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
9,Grogol Petamburan,0.00,0.000000,0.00,0.00,0.020000,0.01,0.000000,0.00,0.00,...,0.000000,0.01,0.000000,0.00,0.00,0.000000,0.01,0.000000,0.01,0.000000


### Create a new DataFrame for Coffee Shop and Cafe data only

In [25]:
tempat_ngops_jkt = jkt_grouped[["Neighborhoods","Coffee Shop", "Café"]]
tempat_ngops_jkt

,Neighborhoods,Coffee Shop,Café
0,Cakung,0.032258,0.000000
1,Cempaka Putih,0.085366,0.048780
2,Cengkareng,0.045455,0.045455
3,Cilandak,0.160000,0.060000
4,Cilincing,0.000000,0.142857
5,Cipayung,0.000000,0.000000
6,Ciracas,0.096774,0.000000
7,Duren Sawit,0.025000,0.012500
8,"Gambir, Jakarta",0.100000,0.010000
9,Grogol Petamburan,0.110000,0.030000


### 7. Cluster Neighborhoods

In [26]:
# set number of clusters
kclusters = 3

jkt_clustering = tempat_ngops_jkt.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jkt_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 1, 0, 0, 2, 0, 2, 2], dtype=int32)

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
jkt_merged = tempat_ngops_jkt.copy()

# add clustering labels
jkt_merged["Cluster Labels"] = kmeans.labels_

In [28]:
jkt_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
jkt_merged

,Neighborhood,Coffee Shop,Café,Cluster Labels
0,Cakung,0.032258,0.000000,0
1,Cempaka Putih,0.085366,0.048780,2
2,Cengkareng,0.045455,0.045455,0
3,Cilandak,0.160000,0.060000,1
4,Cilincing,0.000000,0.142857,0
5,Cipayung,0.000000,0.000000,0
6,Ciracas,0.096774,0.000000,2
7,Duren Sawit,0.025000,0.012500,0
8,"Gambir, Jakarta",0.100000,0.010000,2
9,Grogol Petamburan,0.110000,0.030000,2


In [29]:
# merge Jakarta_grouped with Jakarta_data to add latitude/longitude for each neighborhood
jkt_merged = jkt_merged.join(jkt_df.set_index("Neighborhood"), on="Neighborhood")

print(jkt_merged.shape)
jkt_merged.head() # check the last columns!

(44, 6)


,Neighborhood,Coffee Shop,Café,Cluster Labels,Latitude,Longitude
0,Cakung,0.032258,0.000000,0,-6.19623,106.93522
1,Cempaka Putih,0.085366,0.048780,2,-6.17600,106.87061
2,Cengkareng,0.045455,0.045455,0,-6.13060,106.74559
3,Cilandak,0.160000,0.060000,1,-6.29051,106.79491
4,Cilincing,0.000000,0.142857,0,-6.11358,106.94911


In [30]:
# sort the results by Cluster Labels
print(jkt_merged.shape)
jkt_merged.sort_values(["Cluster Labels"], inplace=True)
jkt_merged

(44, 6)


,Neighborhood,Coffee Shop,Café,Cluster Labels,Latitude,Longitude
0,Cakung,0.032258,0.000000,0,-6.196230,106.935220
36,Pulo Gadung,0.040000,0.030000,0,-6.180390,106.898970
35,"Pesanggrahan, South Jakarta",0.017241,0.051724,0,-6.236400,106.754470
32,Pasar Rebo,0.000000,0.000000,0,-6.337690,106.857550
24,"Makasar, Jakarta",0.000000,0.000000,0,-6.285050,106.902420
22,"Koja, Jakarta",0.000000,0.022727,0,-6.127160,106.899190
42,Tanjung Priok,0.030000,0.070000,0,-6.128420,106.859520
12,Jelambar,0.030000,0.030000,0,-6.152890,106.784350
10,Jagakarsa,0.000000,0.038462,0,-6.343080,106.817450
21,"Kembangan, Jakarta",0.000000,0.000000,0,-6.218230,106.737490


### Finally, let's visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jkt_merged['Latitude'], jkt_merged['Longitude'], jkt_merged['Neighborhood'], jkt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

Cluster 0

In [32]:
jkt_merged[jkt_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Café,Cluster Labels,Latitude,Longitude
0,Cakung,0.032258,0.000000,0,-6.19623,106.93522
36,Pulo Gadung,0.040000,0.030000,0,-6.18039,106.89897
35,"Pesanggrahan, South Jakarta",0.017241,0.051724,0,-6.23640,106.75447
32,Pasar Rebo,0.000000,0.000000,0,-6.33769,106.85755
24,"Makasar, Jakarta",0.000000,0.000000,0,-6.28505,106.90242
22,"Koja, Jakarta",0.000000,0.022727,0,-6.12716,106.89919
42,Tanjung Priok,0.030000,0.070000,0,-6.12842,106.85952
12,Jelambar,0.030000,0.030000,0,-6.15289,106.78435
10,Jagakarsa,0.000000,0.038462,0,-6.34308,106.81745
21,"Kembangan, Jakarta",0.000000,0.000000,0,-6.21823,106.73749


Cluster 1

In [33]:
jkt_merged[jkt_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Café,Cluster Labels,Latitude,Longitude
25,Mampang Prapatan,0.15,0.01,1,-6.245220,106.831940
3,Cilandak,0.16,0.06,1,-6.290510,106.794910
33,Pejompongan,0.17,0.03,1,-6.203864,106.804981
30,"Pancoran, South Jakarta",0.13,0.03,1,-6.255480,106.843910
29,Palmerah,0.12,0.01,1,-6.191530,106.795560
39,Setiabudi,0.17,0.01,1,-6.221240,106.825280
14,Kalibata,0.12,0.03,1,-6.259050,106.841900
41,Tanah Abang,0.18,0.02,1,-6.207850,106.810820


Cluster 2

In [34]:
jkt_merged[jkt_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Café,Cluster Labels,Latitude,Longitude
37,Sawah Besar,0.060000,0.010000,2,-6.15741,106.83140
34,Penjaringan,0.080000,0.040000,2,-6.11662,106.77803
38,Senen,0.090000,0.050000,2,-6.19675,106.84848
40,"Taman Sari, Jakarta",0.060000,0.010000,2,-6.15327,106.82357
31,Pasar Minggu,0.070000,0.030000,2,-6.29265,106.82791
1,Cempaka Putih,0.085366,0.048780,2,-6.17600,106.87061
8,"Gambir, Jakarta",0.100000,0.010000,2,-6.17299,106.81571
26,Matraman,0.090000,0.010000,2,-6.19983,106.86268
27,Menteng,0.090000,0.020000,2,-6.20311,106.83798
9,Grogol Petamburan,0.110000,0.030000,2,-6.16777,106.78460


In [3]:
!pip install folium

     |████████████████████████████████| 102kB 10.1MB/s ta 0:00:01
